In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import joblib

In [2]:
df=pd.read_csv(r"C:\Users\ASUS\Documents\Projects\ML\Kubernetes_POD_failure.csv")
print('succesfully Imported')

succesfully Imported


In [3]:
df.describe()

,cpu_usage_cores,memory_usage_bytes,disk_io_read_bytes,disk_io_write_bytes,network_rx_bytes,network_tx_bytes,restart_count,latency_ms,node_cpu_allocatable_cores,node_memory_allocatable_bytes
count,21374.000000,2.137400e+04,2.137400e+04,2.137400e+04,2.137400e+04,2.137400e+04,21374.000000,21374.000000,21374.000000,2.137400e+04
mean,1.051493,4.044578e+09,5.025267e+06,5.009791e+06,2.507683e+06,2.504195e+06,4.986947,250.984410,7.518668,1.872069e+10
std,0.549617,2.283339e+09,2.877100e+06,2.896186e+06,1.450888e+06,1.442473e+06,3.159283,143.735793,5.361930,1.001221e+10
min,0.100000,1.001814e+08,7.190000e+02,3.260000e+02,7.300000e+02,6.130000e+02,0.000000,1.020000,2.000000,8.000000e+09
25%,0.570000,2.057111e+09,2.537852e+06,2.516285e+06,1.240264e+06,1.253546e+06,2.000000,126.712500,4.000000,8.000000e+09
50%,1.050000,4.042875e+09,5.029259e+06,5.019720e+06,2.509009e+06,2.507016e+06,5.000000,251.770000,8.000000,1.600000e+10
75%,1.530000,6.003775e+09,7.523676e+06,7.524751e+06,3.769187e+06,3.747268e+06,8.000000,375.080000,16.000000,3.200000e+10
max,2.000000,7.999542e+09,9.999135e+06,9.999406e+06,4.999916e+06,4.999974e+06,10.000000,499.970000,16.000000,3.200000e+10


In [4]:
df.head()

,timestamp,namespace,pod_name,node_name,failure_type,failure_injected,failure_detected,cpu_usage_cores,memory_usage_bytes,disk_io_read_bytes,...,network_rx_bytes,network_tx_bytes,oom_killed,restart_count,latency_ms,container_ready,pod_scheduled,node_cpu_allocatable_cores,node_memory_allocatable_bytes,will_fail_soon
0,2025-03-24T06:12:33.224517Z,default,db-deployment-4129,node-09,node_reboot,False,False,1.84,3443305409,6694694,...,188115,4510474,False,3,324.40,True,True,4,16000000000,False
1,2025-03-20T04:34:33.224636Z,kube-system,api-deployment-8335,node-10,pod_stuck,False,False,0.52,5211919506,6468454,...,524949,2571176,False,4,470.15,True,True,8,16000000000,False
2,2025-03-17T18:52:33.224666Z,prod,redis-deployment-7613,node-10,pod_stuck,True,False,1.42,7155712417,6221926,...,4427064,3944211,False,8,61.56,True,True,4,16000000000,False
3,2025-03-23T04:04:33.224689Z,dev,redis-deployment-1489,node-08,filesystem_error,False,False,1.67,4128050631,7563231,...,2065145,1172548,False,3,4.68,False,True,4,16000000000,False
4,2025-03-30T00:16:33.224711Z,prod,api-deployment-1108,node-02,image_pull_error,False,False,0.30,383038696,3189873,...,4150167,2302385,False,6,411.70,True,True,8,16000000000,False


In [5]:
df_reduced = df.drop(["timestamp","pod_name","namespace","node_name"], axis=1)

In [6]:
df_reduced.head()

,failure_type,failure_injected,failure_detected,cpu_usage_cores,memory_usage_bytes,disk_io_read_bytes,disk_io_write_bytes,network_rx_bytes,network_tx_bytes,oom_killed,restart_count,latency_ms,container_ready,pod_scheduled,node_cpu_allocatable_cores,node_memory_allocatable_bytes,will_fail_soon
0,node_reboot,False,False,1.84,3443305409,6694694,4718697,188115,4510474,False,3,324.40,True,True,4,16000000000,False
1,pod_stuck,False,False,0.52,5211919506,6468454,7688454,524949,2571176,False,4,470.15,True,True,8,16000000000,False
2,pod_stuck,True,False,1.42,7155712417,6221926,764623,4427064,3944211,False,8,61.56,True,True,4,16000000000,False
3,filesystem_error,False,False,1.67,4128050631,7563231,715975,2065145,1172548,False,3,4.68,False,True,4,16000000000,False
4,image_pull_error,False,False,0.30,383038696,3189873,6075068,4150167,2302385,False,6,411.70,True,True,8,16000000000,False


In [7]:
le=LabelEncoder()
df_reduced['failure_type']=le.fit_transform(df_reduced["failure_type"])

for col in ["failure_injected", "failure_detected","oom_killed","container_ready","pod_scheduled","will_fail_soon"]:
    df_reduced[col] = le.fit_transform(df_reduced[col])

In [9]:
df_reduced.head()

,failure_type,failure_injected,failure_detected,cpu_usage_cores,memory_usage_bytes,disk_io_read_bytes,disk_io_write_bytes,network_rx_bytes,network_tx_bytes,oom_killed,restart_count,latency_ms,container_ready,pod_scheduled,node_cpu_allocatable_cores,node_memory_allocatable_bytes,will_fail_soon
0,10,0,0,1.84,3443305409,6694694,4718697,188115,4510474,0,3,324.40,1,0,4,16000000000,0
1,13,0,0,0.52,5211919506,6468454,7688454,524949,2571176,0,4,470.15,1,0,8,16000000000,0
2,13,1,0,1.42,7155712417,6221926,764623,4427064,3944211,0,8,61.56,1,0,4,16000000000,0
3,4,0,0,1.67,4128050631,7563231,715975,2065145,1172548,0,3,4.68,0,0,4,16000000000,0
4,6,0,0,0.30,383038696,3189873,6075068,4150167,2302385,0,6,411.70,1,0,8,16000000000,0


In [10]:
X=df_reduced.drop('will_fail_soon',axis=1)
y=df_reduced['will_fail_soon']

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

Training different models to compare their accuracy,precision score and other metrics

In [12]:
models={
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

In [14]:
results={}
for name,model in models.items():
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    results[name] = {"accuracy": acc, "f1_score": f1}
    print(f"\n{name}")
    print(f"Accuracy: {acc*100:.2f}%")
    print(f"F1 Score: {f1:.4f}")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Logistic Regression
Accuracy: 63.67%
F1 Score: 0.0000
              precision    recall  f1-score   support

           0       0.64      1.00      0.78      2722
           1       0.00      0.00      0.00      1553

    accuracy                           0.64      4275
   macro avg       0.32      0.50      0.39      4275
weighted avg       0.41      0.64      0.50      4275

Confusion Matrix:
 [[2722    0]
 [1553    0]]


C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Random Forest
Accuracy: 95.81%
F1 Score: 0.9455
              precision    recall  f1-score   support

           0       1.00      0.93      0.97      2722
           1       0.90      1.00      0.95      1553

    accuracy                           0.96      4275
   macro avg       0.95      0.97      0.96      4275
weighted avg       0.96      0.96      0.96      4275

Confusion Matrix:
 [[2543  179]
 [   0 1553]]

Gradient Boosting
Accuracy: 95.79%
F1 Score: 0.9452
              precision    recall  f1-score   support

           0       1.00      0.93      0.97      2722
           1       0.90      1.00      0.95      1553

    accuracy                           0.96      4275
   macro avg       0.95      0.97      0.96      4275
weighted avg       0.96      0.96      0.96      4275

Confusion Matrix:
 [[2543  179]
 [   1 1552]]


In [15]:
print(results)

{'Logistic Regression': {'accuracy': 0.6367251461988304, 'f1_score': 0.0}, 'Random Forest': {'accuracy': 0.9581286549707603, 'f1_score': 0.945509893455099}, 'Gradient Boosting': {'accuracy': 0.9578947368421052, 'f1_score': 0.9451887941534713}}


Selecting the best model for the prediction

Since in this case false negatives can be costly we are selecting the model based on the f1 scores such that false negatives will be low

In [21]:
best_model_name = max(results, key=lambda k: results[k]["f1_score"])
print(best_model_name)
best_model=models[best_model_name]

Random Forest


Exporting the model

In [23]:
joblib.dump(best_model, "best_pod_failure_model.pkl")
joblib.dump(X.columns.tolist(), "model_features.pkl")
print("Model saved in the system")

Model saved in the system
